In [1]:
import os
import json
import pandas as pd
import numpy as np

from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import svm

from pmlb import fetch_data

## Read Data
You can change the code below to read your own data.

In [3]:
data_name = 'music_origin'

# Returns a pandas DataFrame
df = fetch_data('4544_GeographicalOriginalofMusic')
print(df.describe())

                V1           V2           V3           V4           V5  \
count  1059.000000  1059.000000  1059.000000  1059.000000  1059.000000   
mean     -0.019312    -0.021095    -0.029824    -0.041360     0.025364   
std       0.987911     0.986098     0.972813     0.984577     0.999363   
min      -1.529463    -1.476560    -1.133667    -3.222673    -3.386389   
25%      -0.607616    -0.621808    -0.599229    -0.640634    -0.599677   
50%      -0.184135    -0.211257    -0.301310    -0.134986     0.157396   
75%       0.312155     0.310377     0.158688     0.459267     0.735871   
max       9.439499    10.719304     6.157847     5.926862     2.280230   

                V6           V7           V8           V9          V10  ...  \
count  1059.000000  1059.000000  1059.000000  1059.000000  1059.000000  ...   
mean      0.003521    -0.021183     0.010021    -0.030965    -0.017495  ...   
std       1.000305     0.989127     1.007714     1.007447     1.017138  ...   
min      -4.36668

In [4]:
df.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V110,V111,V112,V113,V114,V115,V116,V117,V118,target
0,7.161286,7.835325,2.911583,0.984049,-1.499546,-2.094097,0.576000,-1.205671,1.849122,-0.425598,...,-0.364194,-0.364194,-0.364194,-0.364194,-0.364194,-0.364194,-0.364194,-15.750000,-47.950001,-0.174878
1,0.225763,-0.094169,-0.603646,0.497745,0.874036,0.290280,-0.077659,-0.887385,0.432062,-0.093963,...,0.936616,0.936616,0.936616,0.936616,0.936616,0.936616,0.936616,14.910000,-23.510000,-0.157189
2,-0.692525,-0.517801,-0.788035,1.214351,-0.907214,0.880213,0.406899,-0.694895,-0.901869,-1.701574,...,0.603755,0.603755,0.603755,0.603755,0.603755,0.603755,0.603755,12.650000,-8.000000,2.718442
3,-0.735562,-0.684055,2.058215,0.716328,-0.011393,0.805396,1.497982,0.114752,0.692847,0.052377,...,0.187169,0.187169,0.187169,0.187169,0.187169,0.187169,0.187169,9.030000,38.740002,-1.020687
4,0.570272,0.273157,-0.279214,0.083456,1.049331,-0.869295,-0.265858,-0.401676,-0.872639,1.147483,...,1.620715,1.620715,1.620715,1.620715,1.620715,1.620715,1.620715,34.029999,-6.850000,-0.190488


In [5]:
df['target'].median(), df['target'].min(), df['target'].max(),

(-0.19119299948215485, -1.505952000617981, 5.854702949523926)

In [6]:
df['target'].value_counts()

 0.209504    2
-0.659703    2
-0.992180    1
 0.074053    1
-0.397769    1
            ..
-0.732467    1
-1.053299    1
 0.673133    1
-0.863037    1
-0.275391    1
Name: target, Length: 1057, dtype: int64

In [7]:
gt = (df['target'] > df['target'].median()).astype(int)
df['gt'] = gt

In [8]:
df['gt'].value_counts()

0    530
1    529
Name: gt, dtype: int64

In [14]:
df.shape

(1059, 119)

## Train a classification model
In this step, you can either train a model in the cell below, or skip the training part and read the model prediction of the training data from your own model .

In [9]:
'''prepare data'''
target_col = 'gt'

to_keep = df.columns[:-2]
X = df[to_keep].values
y = df[target_col].values
y = y.reshape(len(y))

train, test, train_labels, test_labels = train_test_split(X, y, test_size = 0.2)

train_df = pd.DataFrame(train, columns=df.columns[:-2].values)
test_df = pd.DataFrame(test, columns=df.columns[:-2].values)

'''train the model'''
clf = MLPClassifier(random_state=1, max_iter=350)
clf.fit(train, train_labels)

'''report accuracy'''
print(clf.score(test,test_labels ))

0.8066037735849056


/Users/junyuan/opt/anaconda3/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (350) reached and the optimization hasn't converged yet.
  warnings.warn(


In [11]:
y_pred = clf.predict(train)

## Output Training Data

In [12]:
'''
cols: the column names of the input data.
data: the training input.
target_names: the names for the target classes (grount truth).
real_min: minimal values for columns of the input data.
real_max: maximal values for columns of the input data.
y_pred: the prediction of the input data from the model you want to explain.
y_gt: the ground truth of the input data.
'''

def output_data(cols, data, target_names, real_min, real_max, y_pred, y_gt):
    filename = "./output/"+data_name+"/test.json"
    directory = os.path.dirname(filename)
    if not os.path.exists(directory):
        os.makedirs(directory)
        
    to_output = {}
    to_output['columns'] = cols
    to_output['data'] = data
    to_output['target_names'] = target_names
    to_output['real_min'] = real_min
    to_output['real_max'] = real_max
    to_output['y_pred'] = y_pred
    to_output['y_gt'] = y_gt
    with open(filename, 'w') as output:
        output.write(json.dumps(to_output))

In [13]:
''' name the target classes '''
target_names = ["low", "high"]

''' obtain the min and max values '''
min_val = np.min(X, axis=0)
max_val = np.max(X, axis=0)

''' output data '''
output_data(cols=to_keep.tolist(), 
            data=train.tolist(), 
            target_names=target_names, 
            real_min=min_val.tolist(), 
            real_max=max_val.tolist(),
            y_pred=y_pred.tolist(), 
            y_gt=train_labels.tolist())